In [5]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

NameError: name 'tf' is not defined

In [4]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()
#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'Train',str(i))
    images = os.listdir(path)
    for a in images:
        image = Image.open(path + '/'+ a)
        # image = load_img(path + '/'+ a, target_size=(224, 224))
        # image = img_to_array(image)
        image = image.resize((224,224))
        image = np.array(image)
        #sim = Image.fromarray(image)
        if image.shape[2]<4:
          data.append(image)
          labels.append(i)
          # image = np.expand_dims(image, 0)
          # image = imagenet_utils.preprocess_input(image)
    
          # list_image.append(image)
        
#Converting lists into numpy arrays
# list_image = np.vstack(list_image)
labels = np.array(labels)

data = np.array(data)

print(data.shape, labels.shape)

(39254, 224, 224, 3) (39254,)


In [6]:
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


KeyboardInterrupt: 

In [5]:
# augmentation cho training data
aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# augementation cho test
aug_test= ImageDataGenerator(rescale=1./255)

In [5]:
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [6]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Xây thêm các layer
# Lấy output của ConvNet trong VGG16
fcHead = baseModel.output

# Flatten trước khi dùng FCs
fcHead = Flatten(name='flatten')(fcHead)

# Thêm FC
fcHead = Dense(256, activation='relu')(fcHead)
fcHead = Dropout(0.5)(fcHead)

# Output layer với softmax activation
fcHead = Dense(43, activation='softmax')(fcHead)

# Xây dựng model bằng việc nối ConvNet của VGG16 và fcHead
model = model = Model(inputs=baseModel.input, outputs=fcHead)

In [ ]:
# #Building the model
# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))#trich xuat cac dac trung
# model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))#chon cac dac trung nhat
# model.add(Dropout(rate=0.25))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Flatten()) #Bien ma tran thanh vector
# model.add(Dense(256, activation='relu')) # ham kich hoat relu
# model.add(Dropout(rate=0.5))
# model.add(Dense(43, activation='softmax'))#Ham ket qua 100%, tong cac ket qua bang 1

#freeze VGG model
for layer in baseModel.layers:
    layer.trainable = False
    
opt = RMSprop(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
# numOfEpoch = 2
# H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
#                         steps_per_epoch=len(X_train)//32,
#                         validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
#                         validation_steps=len(X_test)//32,
#                         epochs=numOfEpoch)


#Compilation of the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

filepath="weights.best.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

epochs = 100
history = model.fit(X_train, y_train, 
                    validation_split=0.33, 
                    batch_size=32, 
                    epochs=epochs, 
                    validation_data=(X_test, y_test), 
                    callbacks=callbacks_list,
                    verbose=0
                   )


In [31]:
model.save("my_model.h5")

In [32]:


#plotting graphs for accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined

<Figure size 432x288 with 0 Axes>

In [33]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))

X_test=np.array(data)

pred = model.predict_classes(X_test)

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))


AttributeError: 'Functional' object has no attribute 'predict_classes'